In [23]:
import pandas as pd
import json
import datetime

In [13]:
admin_config_path = "../dummy_test_cfg.json"
with open(admin_config_path, "r") as fd:
    admin_config = json.load(fd)

In [17]:
dataframe_path = admin_config['dataframe_path']
dataframe = pd.read_pickle('../' + dataframe_path)

In [20]:
dataframe.head()

,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,BND_DAYTIME,BND_CLEARSKY_GHI,TBL_DAYTIME,TBL_CLEARSKY_GHI,DRA_DAYTIME,DRA_CLEARSKY_GHI,FPK_DAYTIME,FPK_CLEARSKY_GHI,GWN_DAYTIME,GWN_CLEARSKY_GHI,PSU_DAYTIME,PSU_CLEARSKY_GHI,SXF_DAYTIME,SXF_CLEARSKY_GHI
iso-datetime,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,nan,/project/cq-training-1/project1/data/hdf5v7_8b...,64,/project/cq-training-1/project1/data/hdf5v5_16...,64,0,0.0,0,0.0,1,52.262615,0,0.0,0,0.0,0,0.0,0,0.0
2015-01-01 00:15:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,65,/project/cq-training-1/project1/data/hdf5v5_16...,65,0,0.0,0,0.0,1,18.440082,0,0.0,0,0.0,0,0.0,0,0.0
2015-01-01 00:30:00,nan,/project/cq-training-1/project1/data/hdf5v7_8b...,66,/project/cq-training-1/project1/data/hdf5v5_16...,66,0,0.0,0,0.0,1,1.445237,0,0.0,0,0.0,0,0.0,0,0.0
2015-01-01 00:45:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,67,/project/cq-training-1/project1/data/hdf5v5_16...,67,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0
2015-01-01 01:00:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,68,/project/cq-training-1/project1/data/hdf5v5_16...,68,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0


In [44]:
target_datetimes = [datetime.datetime.fromisoformat(d) for d in admin_config["target_datetimes"]]

In [45]:
admin_config

{'dataframe_path': 'dummy_test_catalog.pkl',
 'start_bound': '2015-01-01',
 'end_bound': '2016-01-01',
 'stations': {'BND': [40.05192, -88.37309, 230],
  'TBL': [40.12498, -105.2368, 1689],
  'DRA': [36.62373, -116.01947, 1007],
  'FPK': [48.30783, -105.1017, 634],
  'GWN': [34.2547, -89.8729, 98],
  'PSU': [40.72012, -77.93085, 376],
  'SXF': [43.73403, -96.62328, 473]},
 'target_time_offsets': ['P0DT0H0M0S',
  'P0DT1H0M0S',
  'P0DT3H0M0S',
  'P0DT6H0M0S'],
 'target_datetimes': ['2015-01-01T12:45:00',
  '2015-01-01T19:00:00',
  '2015-01-02T13:30:00',
  '2015-01-02T19:45:00',
  '2015-01-03T13:45:00',
  '2015-01-03T20:00:00',
  '2015-01-04T14:30:00',
  '2015-01-04T20:45:00',
  '2015-01-05T14:45:00',
  '2015-01-05T21:15:00',
  '2015-01-06T16:00:00',
  '2015-01-06T22:15:00',
  '2015-01-07T23:00:00',
  '2015-01-08T17:15:00',
  '2015-01-08T23:15:00',
  '2015-01-09T17:30:00',
  '2015-01-09T23:30:00',
  '2015-01-10T17:45:00',
  '2015-01-10T23:45:00',
  '2015-01-11T18:15:00',
  '2015-01-12T00:

In [48]:
# station = [*stations][0]
station = 'DRA'
for dt in target_datetimes:
    print(dataframe.lookup([dt],[station + '_CLEARSKY_GHI']))

[0.]
[517.24665383]
[0.]
[539.1543395]
[0.]
[539.93283053]
[0.]
[515.20006526]
[0.]
[480.5100123]
[125.36949148]
[365.96524358]
[249.26023062]
[343.52787153]
[208.18514921]
[381.65519266]
[165.77988676]
[417.09708419]
[122.87787697]
[476.84345057]
[39.98518217]
[502.78748144]
[0.20270509]
[541.54326314]
[0.]
[556.7674025]
[0.]
[568.31802193]
[0.]
[576.13087954]
[0.]
[580.15885945]
[0.]
[580.37204652]
[0.]
[576.75791544]
[0.]
[534.34475889]
[17.45541145]
[515.43858126]
[98.83549214]
[492.95398143]
[100.9630857]
[497.49511454]
[103.21097952]
[502.10053064]
[22.16273534]
[533.64161636]
[0.]
[561.45593977]
[0.]
[566.20850367]
[0.]
[605.38722667]
[0.]
[621.30440233]
[0.]
[633.09323789]
[0.]
[640.70871336]
[0.]
[637.43746428]


In [49]:
len(target_datetimes)

62